In [5]:
import pandas as pd
import numpy as np
import os

from sklearn.preprocessing import MinMaxScaler
import sklearn
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import joblib

def sort_csv_files(folder_path):
    files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    dfs = []
    for file in files:
        df = pd.read_csv(os.path.join(folder_path, file))
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

def filter_valfonc(df):
    mutation = df
    mutation = mutation[~mutation["valeurfonc"].isna()]
    mutation = mutation[mutation["valeurfonc"] >= 1000]
    mutation = mutation[mutation["valeurfonc"] <= 1000000]
    return mutation

def add_arrondissement(df):
    n_col = df["l_codinsee"].apply(lambda s: s[-4:-2] if s[2:4]=="75" else 0)
    #n_col = n_col.rename("arro")
    df["arro"] = n_col.astype("int")

def sum_surface(df):
    df["surf"] = (df["sbati"] 
                  + df["sbatmai"] 
                  + df["sbatapt"]
                  + df["sbatact"])
    
def min_max_scale(df):
    scaler = StandardScaler()
    df['valeurfonc'] = scaler.fit_transform(df['valeurfonc'].values.reshape(-1, 1))

def X_y_split(df):
    features = ["anneemut", "moismut", "nblot", "sterr", "nbvolmut", "nblocmut",
                "nblocmai", "nblocapt", "nblocdep", "nblocact", "latitude", "longitude",
                "arro", "surf"
               ]
    X = df[features]
    y = df["valeurfonc"]
    return X, y

def training(X_train, Y_train):  
    xgboost = XGBRegressor()
    xgboost.fit(X_train, Y_train)

def predict(X_test):
    y_pred = xgboost.predict(X_test)
    return y_pred

def save_model(pkl_file_path):
    joblib.dump(xgboost, pkl_file_path)
    
def load_model(pkl_file_path):
    xgboost = joblib.load(pkl_file_path)
    return xgboost

In [6]:
from time import time

start = time()

path = "data_right_price/data_localisee/"
df_1 = sort_csv_files(path)
mutation = filter_valfonc(df_1)
add_arrondissement(mutation)
sum_surface(mutation)
min_max_scale(mutation)
X, y = X_y_split(mutation)
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.2, random_state = 100)

/var/folders/s5/c05fkgj90cz2t3tmp495bdfc0000gn/T/ipykernel_64983/2506388647.py:6: DtypeWarning: Columns (60) have mixed types.Specify dtype option on import or set low_memory=False.
  df_1 = sort_csv_files(path)


In [7]:
training(X_train, Y_train)
y_pred = predict(X_test)

print(f"{round(time()-start, 2)}sec")

NameError: name 'xgboost' is not defined